In [ ]:
import gc
import copy
import math
import random
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm
from typing import Dict, Any
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from modules.labeling_lib import get_prices, get_labels_one_direction, get_features
from modules.tester_lib import test_model_one_direction
import optuna
from optuna.pruners import SuccessiveHalvingPruner, HyperbandPruner
from modules.export_lib import export_model_to_ONNX
import warnings
warnings.filterwarnings("ignore")
optuna.logging.set_verbosity(optuna.logging.WARNING)

def bootstrap_oob_identification(X, y, n_models=25):
    """Identifica muestras problemáticas usando bootstrap OOB"""
    # Contadores para errores
    oob_counts = pd.Series(0, index=X.index)
    error_counts_0 = pd.Series(0, index=X.index)
    error_counts_1 = pd.Series(0, index=X.index)
    
    for _ in range(n_models):
        # Bootstrap sample
        frac_bootstrap = random.uniform(0.4, 0.6)
        train_idx = X.sample(frac=frac_bootstrap, replace=True, random_state=None).index
        val_idx = X.index.difference(train_idx)
        
        if len(val_idx) == 0:
            continue
            
        # Entrenar modelo
        model = CatBoostClassifier(
            iterations=random.randint(100, 500),
            depth=random.randint(3, 10),
            learning_rate=random.uniform(0.1, 0.5),
            l2_leaf_reg=random.uniform(0.0, 1.0),
            verbose=False
        )
        model.fit(X.loc[train_idx], y.loc[train_idx])
        
        # Predicciones OOB
        pred_proba = model.predict_proba(X.loc[val_idx])[:, 1]
        pred_labels = (pred_proba >= 0.5).astype(int)
        
        # Encontrar diferencias para cada clase
        val_y = y.loc[val_idx]
        val_0_idx = val_idx[val_y == 0]
        val_1_idx = val_idx[val_y == 1]
        
        diff_0 = val_0_idx[pred_labels[val_y == 0] != 0]
        diff_1 = val_1_idx[pred_labels[val_y == 1] != 1]
        
        oob_counts.loc[val_idx] += 1
        error_counts_0.loc[diff_0] += 1
        error_counts_1.loc[diff_1] += 1
    
    return error_counts_0, error_counts_1, oob_counts

def optimize_bad_samples_threshold(error_counts_0, error_counts_1, oob_counts, fractions_to_try=[0.5, 0.6, 0.7, 0.8]):
    """Optimiza el umbral para identificar muestras problemáticas"""
    to_mark_0 = (error_counts_0 / oob_counts.replace(0, 1)).fillna(0)
    to_mark_1 = (error_counts_1 / oob_counts.replace(0, 1)).fillna(0)
    
    best_fraction = None
    best_score = np.inf
    
    for frac in fractions_to_try:
        # Calcular umbrales
        threshold_0 = np.percentile(to_mark_0[to_mark_0 > 0], 75) * frac if len(to_mark_0[to_mark_0 > 0]) else 0
        threshold_1 = np.percentile(to_mark_1[to_mark_1 > 0], 75) * frac if len(to_mark_1[to_mark_1 > 0]) else 0
        
        # Marcar muestras problemáticas
        marked_0 = to_mark_0[to_mark_0 > threshold_0].index
        marked_1 = to_mark_1[to_mark_1 > threshold_1].index
        all_bad = pd.Index(marked_0).union(marked_1)
        
        # Calcular score (media de error en muestras buenas)
        good_mask = ~to_mark_0.index.isin(all_bad)
        error_ratios_good = []
        for idx in to_mark_0[good_mask].index:
            if to_mark_0[idx] > 0:
                error_ratios_good.append(to_mark_0[idx])
            if to_mark_1[idx] > 0:
                error_ratios_good.append(to_mark_1[idx])
        
        mean_error_good = np.mean(error_ratios_good) if error_ratios_good else 1.0
        
        if mean_error_good < best_score:
            best_score = mean_error_good
            best_fraction = frac
    
    return best_fraction, best_score

def objective(trial: optuna.trial.Trial, base_hp: dict, study=None) -> float:
    def calc_score(fwd: float, bwd: float, eps: float = 1e-9) -> float:
        if (fwd is None or bwd is None or
            not np.isfinite(fwd) or not np.isfinite(bwd) or
            fwd <= 0 or bwd <= 0):
            return -1.0
        mean = 0.5 * (fwd + bwd)
        delta = abs(fwd - bwd) / max(abs(fwd), abs(bwd), eps)
        score = mean * (1.0 - delta)
        return score

    best_combined_score = -math.inf
    hp = {k: copy.deepcopy(v) for k, v in base_hp.items() if k != 'base_df'}
    gc.collect()
    
    # Optimización de períodos para el modelo principal
    n_periods = trial.suggest_int('n_periods', 5, 15)
    periods = []
    for i in range(n_periods):
        period = trial.suggest_int(f'period_{i}', 5, 200, log=True)
        periods.append(period)
    hp['periods_main'] = sorted(list(set(periods)))

    # Selección de estadísticas para el modelo principal
    stat_choices = [
        "std", "skew", "kurt", "zscore", "range", "mad", "entropy", 
        "slope", "momentum", "fractal", "hurst", "autocorr", "max_dd", 
        "sharpe", "fisher", "chande", "var", "approx_entropy", 
        "eff_ratio", "corr_skew", "jump_vol", "vol_skew"
    ]
    stats = trial.suggest_int('stats_main', 1, 5)
    selected_main_stats = []
    for i in range(stats):
        stat = trial.suggest_categorical(f'stat_{i}', stat_choices)
        selected_main_stats.append(stat)
    selected_main_stats = list(set(selected_main_stats))
    if len(selected_main_stats) == 1 and ("fractal" in selected_main_stats or "hurst" in selected_main_stats or "kurt" in selected_main_stats):
        remaining_stats = [s for s in stat_choices if s != "fractal" and s != "hurst" and s != "kurt"]
        additional_stat = trial.suggest_categorical('additional_stat', remaining_stats)
        selected_main_stats.append(additional_stat)
    hp["stats_main"] = selected_main_stats

    # Optimización de parámetros de etiquetado
    hp['label_min'] = trial.suggest_int('label_min', 1, 5)
    hp['label_max'] = trial.suggest_int('label_max', hp['label_min']+5, 30)
    hp['markup'] = trial.suggest_float("markup", 0.1, 0.4)

    # CatBoost principal - Mayor capacidad de aprendizaje
    hp['cat_main_iterations'] = trial.suggest_int('cat_main_iterations', 300, 2000, step=100)
    hp['cat_main_depth'] = trial.suggest_int('cat_main_depth', 6, 12)
    hp['cat_main_learning_rate'] = trial.suggest_float('cat_main_learning_rate', 0.005, 0.4, log=True)
    hp['cat_main_l2_leaf_reg'] = trial.suggest_float('cat_main_l2_leaf_reg', 0.5, 10.0)
    hp['cat_main_early_stopping'] = trial.suggest_int('cat_main_early_stopping', 10, 50, step=10)
    #hp['cat_main_rsm'] = trial.suggest_float('cat_main_rsm', 0.1, 1.0)

    # CatBoost meta - Enfoque en precisión
    hp['cat_meta_iterations'] = trial.suggest_int('cat_meta_iterations', 200, 1000, step=100)
    hp['cat_meta_depth'] = trial.suggest_int('cat_meta_depth', 5, 10)
    hp['cat_meta_learning_rate'] = trial.suggest_float('cat_meta_learning_rate', 0.01, 0.3, log=True)
    hp['cat_meta_l2_leaf_reg'] = trial.suggest_float('cat_meta_l2_leaf_reg', 0.5, 8.0)
    hp['cat_meta_early_stopping'] = trial.suggest_int('cat_meta_early_stopping', 10, 50, step=10)

    # Optimización del número de meta-learners y fracción de muestras malas
    hp['n_meta_learners'] = trial.suggest_int('n_meta_learners', 1, 10)
    hp['bad_samples_fraction'] = trial.suggest_float('bad_samples_fraction', 0.5, 1.0)

    # Obtener datos y características
    full_ds = get_labels_one_direction(get_features(get_prices(hp), hp), 
                                     markup=hp['markup'], 
                                     min=hp['label_min'],
                                     max=hp['label_max'],
                                     direction=hp['direction'])

    # Dividir en períodos de entrenamiento, backward testing y forward testing
    train_mask = (full_ds.index > hp['backward']) & (full_ds.index < hp['forward'])
    oos_mask = (full_ds.index >= hp['forward']) & (full_ds.index < hp['full_forward'])

    # Extraer características y asegurar el orden consistente
    feature_cols = sorted(full_ds.filter(regex='_feature$').columns)
    ds_train = full_ds.loc[train_mask, feature_cols]
    y_train = full_ds.loc[train_mask]['labels'].astype('int16')
    ds_oos = full_ds.loc[oos_mask, feature_cols]
    ds_train_val = ds_train.iloc[-len(ds_oos):]

    # Clustering con ventana deslizante
    assert ds_train.index.max() < hp['forward'], \
       "¡ds_train contiene fechas posteriores al límite forward!"
    assert ds_oos.index.min() >= hp['forward'], \
        "¡ds_oos incluye datos dentro del tramo backward!"

    # Identificación de muestras problemáticas
    error_counts_0, error_counts_1, oob_counts = bootstrap_oob_identification(
        ds_train, 
        y_train,
        n_models=hp['n_meta_learners']
    )
    best_fraction, best_score = optimize_bad_samples_threshold(
        error_counts_0, 
        error_counts_1, 
        oob_counts
    )
    # Marcar muestras problemáticas con el mejor umbral
    to_mark_0 = (error_counts_0 / oob_counts.replace(0, 1)).fillna(0)
    to_mark_1 = (error_counts_1 / oob_counts.replace(0, 1)).fillna(0)
    threshold_0 = np.percentile(to_mark_0[to_mark_0 > 0], 75) * best_fraction if len(to_mark_0[to_mark_0 > 0]) else 0
    threshold_1 = np.percentile(to_mark_1[to_mark_1 > 0], 75) * best_fraction if len(to_mark_1[to_mark_1 > 0]) else 0
    marked_0 = to_mark_0[to_mark_0 > threshold_0].index
    marked_1 = to_mark_1[to_mark_1 > threshold_1].index
    all_bad = pd.Index(marked_0).union(marked_1)
    # Crear etiquetas meta
    ds_train['meta_labels'] = 1.0
    ds_train.loc[ds_train.index.isin(all_bad), 'meta_labels'] = 0.0

    # Entrenar modelo principal solo con muestras buenas
    good_samples = ds_train[ds_train['meta_labels'] == 1.0]
    X_main = good_samples[feature_cols]
    y_main = y_train[good_samples.index]
    X_train, X_val, y_train_split, y_val = train_test_split(
        X_main, y_main, train_size=0.8, test_size=0.2, shuffle=True)
    cat_main_params = dict(
        iterations=hp.get('cat_main_iterations', 500),
        depth=hp.get('cat_main_depth', 6),
        learning_rate=hp.get('cat_main_learning_rate', 0.15),
        l2_leaf_reg=hp.get('cat_main_l2_leaf_reg', 3.0),
        early_stopping_rounds=hp.get('cat_main_early_stopping'),
        #rsm=hp.get('cat_main_rsm', 0.5),
        custom_loss=['Accuracy'],
        eval_metric='Accuracy',
        use_best_model=True,
        verbose=False,
        thread_count=-1,
        task_type='CPU',
    )
    main_model = CatBoostClassifier(**cat_main_params)
    main_model.fit(X_train, y_train_split, eval_set=(X_val, y_val), plot=False)

    # Entrenar modelo meta con todos los datos
    X_meta = ds_train[feature_cols]
    y_meta = ds_train['meta_labels']
    X_meta_train, X_meta_val, y_meta_train, y_meta_val = train_test_split(
        X_meta, y_meta, train_size=0.8, test_size=0.2, shuffle=True)
    cat_meta_params = dict(
        iterations=hp.get('cat_meta_iterations', 500),
        depth=hp.get('cat_meta_depth', 6),
        learning_rate=hp.get('cat_meta_learning_rate', 0.15),
        l2_leaf_reg=hp.get('cat_meta_l2_leaf_reg', 3.0),
        early_stopping_rounds=hp.get('cat_meta_early_stopping'),
        custom_loss=['F1'],
        eval_metric='F1',
        use_best_model=True,
        verbose=False,
        thread_count=-1,
        task_type='CPU',
    )
    meta_model = CatBoostClassifier(**cat_meta_params)
    meta_model.fit(X_meta_train, y_meta_train, eval_set=(X_meta_val, y_meta_val), plot=False)

    # Evaluar en datos fuera de muestra
    ds_oos['close'] = full_ds.loc[oos_mask, 'close']
    R2_forward = test_model_one_direction(ds_oos,
                                         [main_model, meta_model],
                                         hp['full_forward'],
                                         hp['backward'],
                                         hp['markup'],
                                         hp['direction'],
                                         plt=False)

    # Evaluar en datos históricos
    ds_train_val['close'] = full_ds.loc[ds_train.iloc[-len(ds_oos):].index, 'close']
    R2_backward = test_model_one_direction(ds_train_val,
                                         [main_model, meta_model],
                                         hp['forward'],
                                         hp['backward'],
                                         hp['markup'],
                                         hp['direction'],
                                         plt=False)

    if R2_forward > 0.0  and R2_backward > 0.0:
        score = calc_score(R2_forward, R2_backward)
    else:
        score = -10.0
    
    if score > best_combined_score:
        best_combined_score = score
        # Guardar información del trial actual
        trial.set_user_attr("forward_r2", R2_forward)
        trial.set_user_attr("backward_r2", R2_backward)
        trial.set_user_attr("combined_score", score)
        trial.set_user_attr("stats_main", hp["stats_main"])
        trial.set_user_attr("periods_main", hp["periods_main"])
        trial.set_user_attr("n_meta_learners", hp["n_meta_learners"])
        trial.set_user_attr("bad_samples_fraction", hp["bad_samples_fraction"])
        
        # Guardar parámetros del trial actual (sin fechas)
        params_to_save = hp.copy()
        params_to_save.pop('backward', None)
        params_to_save.pop('forward', None)
        params_to_save.pop('full_forward', None)
        trial.set_user_attr("params", params_to_save)
        
        # Si existe el estudio, actualizar sus atributos
        if study is not None:
            current_best = study.user_attrs.get("best_combined_score", -math.inf)
            if score > current_best:
                study.set_user_attr("best_params", params_to_save)
                study.set_user_attr("best_metrics", {
                    "forward_r2": R2_forward,
                    "backward_r2": R2_backward,
                    "combined_score": score,
                })
                study.set_user_attr("best_combined_score", score)
                study.set_user_attr("best_models", [main_model, meta_model])
                study.set_user_attr("best_stats", hp["stats_main"])
                study.set_user_attr("best_periods", hp["periods_main"])
                study.set_user_attr("best_trial_number", trial.number)
                study.set_user_attr("best_df_sample", full_ds.sample(1))

    return score

def optimize_and_export(base_hp: Dict[str, Any]):
    """Lanza Optuna, guarda el mejor modelo y lo exporta a ONNX."""
    try:
        # Crear el estudio
        study = optuna.create_study(
            direction='maximize',
            pruner=HyperbandPruner(),
            sampler=optuna.samplers.TPESampler(
                n_startup_trials=int(np.sqrt(base_hp['n_trials'])),
            )
        )
        
        study.optimize(lambda t: objective(t, base_hp, study),
                    n_trials=base_hp['n_trials'],
                    show_progress_bar=True)

        # Obtener el mejor modelo
        best_models = study.user_attrs.get("best_models")
        if not (best_models and len(best_models) == 2 and all(best_models)):
            print("⚠️  Error: best_models incorrecto")
            return None

        # Exportar modelos CatBoost directamente
        export_params = base_hp.copy()
        export_params.update({
            "best_trial": study.user_attrs["best_trial_number"],
            "best_score": study.user_attrs["best_combined_score"],
            "best_periods": study.user_attrs["best_periods"],
            "best_stats": study.user_attrs["best_stats"],
            "best_stats_meta": study.user_attrs["best_stats_meta"],
            "best_models": best_models,  # Los modelos CatBoost directamente
        })
        
        export_model_to_ONNX(**export_params)
        
        return {
            "forward_r2": study.user_attrs.get("best_metrics", {}).get("forward_r2", float("nan")),
            "backward_r2": study.user_attrs.get("best_metrics", {}).get("backward_r2", float("nan")),
            "combined_score": study.user_attrs.get("best_metrics", {}).get("combined_score", float("nan")),
        }
    except Exception as e:
        print(f"Error en optimize_and_export: {str(e)}")
        import traceback
        print(traceback.format_exc())
        return None

if __name__ == "__main__":
    base_hp: Dict[str, Any] = {
        'symbol': r'XAUUSD',
        'timeframe': 'H1',
        'direction': 'buy',
        'backward': datetime(2020, 2, 1),
        'forward': datetime(2024, 2, 1),
        'full_forward': datetime(2026, 2, 1),
        'model_seed': 0,
        'n_trials': 1000,
        'models_export_path': r'/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/6C3C6A11D1C3791DD4DBF45421BF8028/MQL5/Files/',
        'include_export_path': r'/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/6C3C6A11D1C3791DD4DBF45421BF8028/MQL5/Include/ajmtrz/include/Dmitrievsky',
        'history_path': r"/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/Common/Files/",
        'stats_main': [],
        'best_models': [None, None],
        'markup': 0.20,
        'label_min'  : 1,
        'label_max'  : 15,
        'window_size': 350,
        'periods_main': [i for i in range(5, 300, 30)],
    }
    
    base_df = get_prices(base_hp)
    base_hp.update({
        'base_df': base_df,
    })
    # Para recopilar resultados globales de todos los modelos
    all_results = {}
    best_models = []
    model_range = range(0, 10)
    base_hp.update({
        'n_trials': 500,
    })
    for i in tqdm(model_range, desc=f"Optimizando {base_hp['symbol']}/{base_hp['timeframe']}", unit="modelo"):
        try:
            model_results = optimize_and_export(base_hp)
            best_models.append((i, model_results))
            
            # Añadir a resultados globales
            all_results[f"model_{i}"] = {
                "success": True,
                "forward_r2": model_results["forward_r2"],
                "backward_r2": model_results["backward_r2"],
                "combined_score": model_results["combined_score"]
            }
            
        except Exception as e:
            import traceback
            tqdm.write(f"\nError procesando modelo {i}: {str(e)}")
            tqdm.write(traceback.format_exc())
            
            all_results[f"model_{i}"] = {
                "success": False,
                "error": str(e)
            }
            continue
    
    # Resumen final
    print("\n" + "="*50)
    print(f"RESUMEN DE OPTIMIZACIÓN {base_hp['symbol']}/{base_hp['timeframe']}")
    print("="*50)
    
    successful_models = [info for model_key, info  in all_results.items() if info.get("success", False)]
    print(f"Modelos completados exitosamente: {len(successful_models)}/{len(model_range)}")

    if successful_models:
        # Calcular estadísticas globales
        forward_scores = [info["forward_r2"] for info in successful_models]
        backward_scores = [info["backward_r2"] for info in successful_models]
        combined_scores = [info["combined_score"] for info in successful_models]
        
        print(f"\nEstadísticas de rendimiento:")
        print(f"  Forward R2 promedio: {np.mean(forward_scores):.4f} ± {np.std(forward_scores):.4f}")
        print(f"  Backward R2 promedio: {np.mean(backward_scores):.4f} ± {np.std(backward_scores):.4f}")
        print(f"  Puntuación combinada promedio: {np.mean(combined_scores):.4f} ± {np.std(combined_scores):.4f}")

        # Identificar el mejor modelo global basado en la puntuación combinada
        successful = [(k, v) for k, v in all_results.items() if v.get("success", False)]
        combined_scores = [v["combined_score"] for _, v in successful]
        best_model_key, best_info = successful[int(np.argmax(combined_scores))]
        
        print(f"\nMejor modelo global: {best_model_key}")
        print(f"  Forward R2: {best_info['forward_r2']:.4f}")
        print(f"  Backward R2: {best_info['backward_r2']:.4f}")
        print(f"  Puntuación combinada: {best_info['combined_score']:.4f}")
    
    print("\nProceso de optimización completado.")